In [ ]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import re
import io
from config import api_key_inCites
from config import api_key_WOS
from collections import OrderedDict
from pandas import json_normalize
import base64
import xmltodict
import time 
import concurrent.futures

In [ ]:
##InCites Resources: 
#https://clarivate.com/webofsciencegroup/solutions/xml-and-apis/
#https://developer.clarivate.com/help
#https://api.clarivate.com/api/incites/DocumentLevelMetricsByUT/json
#https://github.com/Clarivate-SAR/incites-retrieve
#https://api.clarivate.com/swagger-ui/?url=https%3A%2F%2Fdeveloper.clarivate.com%2Fapis%2Fincites%2Fswagger%3FforUser%3D9c13dcee882598956b564212f82c2236a51e3f56


In [ ]:
## Upload a .csv from Elements Reporting Database

## "data/2022_03-22 FSM Publications 2017-2022.csv"
## "data/utpractice.csv"

fsm_elements_publications_path = "data/2022_03-22 FSM Publications 2017-2022.csv"

## Read the CSV file and store into Pandas DataFrame with the column Scopus Author ID as a string
fsm_elements_df = pd.read_csv(fsm_elements_publications_path, encoding = "ISO-8859-1", na_values=['NULL', '<NA>'])

#Change the column names to lower case with underscore for spaces
fsm_elements_df.columns =  fsm_elements_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
fsm_elements_df.head()
    


In [ ]:
#The function "prep_UT_list" takes in the UT_list_df dataframe and formats the "id" column 
#as a string, then uses the column to create a list, removes any of the "nan" values lines that don't 
#have an ID, and finally returns a list called "cleaned_UT_list"
    
## Change the data type in the dataframe column called "web_of_science" to a string. 
fsm_elements_df['web_of_science'] = fsm_elements_df['web_of_science'].astype(str)

## Save the column called id to a list called interim_UT_list
interim_UT_list = fsm_elements_df['web_of_science'].tolist()
# print(interim_UT_list)

# Remove nan values from list
interim_UT_list_2 = [x for x in interim_UT_list if str(x) != 'nan']

## Remove duplicates from interim_UT_list
interim_UT_list_3 = list(set(interim_UT_list_2))
# print(interim_UT_list_2)

#Remove the WOS: characters from each item in the list
cleaned_UT_list = [i.replace("WOS:", "") for i in interim_UT_list_3]
# print(cleaned_UT_list)

## Resources
## https://www.geeksforgeeks.org/python-ways-to-remove-duplicates-from-list/


In [ ]:
## Chunk list of identifiers for batch api query

## Create variables
chunks = []
n = 100

## Create for loop with range
## python's range function takes (start, stop, step)

for i in range(0, len(cleaned_UT_list), n):
    
    ## print(i)
    
    ## Reference items in a list by using [start:stop] with numbers
    ## example is:  0 : 0 + 100 = start list at 0 and end list at 0 + 100
    chunk = cleaned_UT_list[i:i + n]
    
    ## Append each list chunk into a larger list called chunks (i.e. a list of lists)
    chunks.append(chunk)
       

## Inspect result
# print(len(chunks))
# print(chunks)


## Resources
## https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

In [ ]:
## Turn each chunk list into a string separated by commas

## Create variables
multiple_strings = []

## Use for loop to access each chunk to turn it into a string
## Essentially creating a list of strings (rather than a list of lists)

for chunk in chunks: 
    
    ## Use python's join method to join items from a list into one string separated by a comma
    single_string = ",".join(chunk)
    
    ## Inspect Result
    #print(type(single_string))
    #print(single_string)
    
    ## Append each new string to a list, to create a list of strings 
    multiple_strings.append(single_string)

## Inspect Result
print(type(multiple_strings))


In [ ]:
## Create API call to InCites using chunks


## Create variables
multiple_chunk_list = []
single_chunk_dict = {}
BATCH_SIZE= 100

## Create for loop 

for item in multiple_strings: 
        
    url = "https://api.clarivate.com/api/incites/DocumentLevelMetricsByUT/json"
    headers = {
            'X-ApiKey': api_key_inCites,
            'Accept':'application/json'
            }

    parameters = {
            'UT' : item,
            'batch': n,
            "ver": 2,
            "schema": "ct", 
            "esci": "y",                  
            }
        
        
    try:
         
        ## Python time method sleep() suspends execution for the given number of seconds 
        time.sleep(0.1) 

        ## Make the API request 
        single_chunk_response = requests.get(url, headers=headers, params=parameters)
        
        ## Inspect result
        #print(single_chunk_response.url)
        #print(single_chunk_response.status_code)

        ## Use the response library's .json() function to returns a JSON object of the api response
        single_chunk_dict = single_chunk_response.json()
        
            
        ## Append each single_chunk_dict to multiple_chunk_list to create a list of dictionaries
        multiple_chunk_list.append(single_chunk_dict.copy())
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print("------------")
    


In [ ]:
## Inspect the result
print(len(multiple_chunk_list))
print(type(multiple_chunk_list))
# print(multiple_chunk_list)

In [ ]:
## Use json_normalize to flatten the json contained in the "api" field. 
 
incites_df = pd.DataFrame.from_dict(json_normalize(multiple_chunk_list, meta=["api"], record_path=["api", "rval"]),orient="columns")
# incites_df.head()


#References
#https://stackoverflow.com/questions/48177934/flatten-or-unpack-list-of-nested-dicts-in-dataframe
#https://stackoverflow.com/questions/50161070/convert-list-of-dicts-of-dict-into-dataframe
#https://stackoverflow.com/questions/43984865/python-having-trouble-returning-a-pandas-data-frame-from-a-user-defined-functio
#https://stackoverflow.com/questions/37668291/flatten-double-nested-json

In [ ]:
## Flattens the json contained in the PERCENTILE field and creates series then columns of the content. The funciton returns a pandas dataframe

remove_percentile_nest = pd.concat([incites_df.drop(['PERCENTILE'], axis=1), incites_df['PERCENTILE'].apply(pd.Series)], axis=1, join="outer")
remove_percentile_nest.rename(columns= {0:'CTMacro', 1:'CTMeso', 2:'CTMicro' }, inplace=True)
# remove_percentile_nest.head()

#References
#https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29330853

In [ ]:
## Flattens the json contained in the CT Macro, Meso, Micro columns and creates series then columns of the content. 
remove_macro_nest = pd.concat([remove_percentile_nest.drop(['CTMacro'], axis=1), remove_percentile_nest['CTMacro'].apply(pd.Series)], axis=1, join="outer")
remove_macro_nest.rename(columns= {"CODE":'CTMacro_Code', 
                                   "CAT_PERC":'CTMacro_Cat_Perc', 
                                   "SUBJECT":'CTMacro_Subj',
                                   "CAT_EXP_CITATION": "CTMacro_CatExpCitation",
                                   "LEVEL": "CTMacro_Level",
                                   "IS_BEST": "CTMacro_IsBest",
                                   "CNCI": "CTMacro_CNCI"
                                  }, inplace=True)
# remove_macro_nest.head()


remove_meso_nest = pd.concat([remove_macro_nest.drop(['CTMeso'], axis=1), remove_macro_nest['CTMeso'].apply(pd.Series)], axis=1, join="outer")
remove_meso_nest.rename(columns= {"CODE":'CTMeso_Code', 
                                   "CAT_PERC":'CTMeso_Cat_Perc', 
                                   "SUBJECT":'CTMeso_Subj',
                                   "CAT_EXP_CITATION": "CTMeso_CatExpCitation",
                                   "LEVEL": "CTMeso_Level",
                                   "IS_BEST": "CTMeso_IsBest",
                                   "CNCI": "CTMeso_CNCI"
                                  }, inplace=True)
# remove_meso_nest.head()

remove_micro_nest = pd.concat([remove_meso_nest.drop(['CTMicro'], axis=1), remove_meso_nest['CTMicro'].apply(pd.Series)], axis=1, join="outer")
remove_micro_nest.rename(columns= {"CODE":'CTMicro_Code', 
                                   "CAT_PERC":'CTMicro_Cat_Perc', 
                                   "SUBJECT":'CTMicro_Subj',
                                   "CAT_EXP_CITATION": "CTMicro_CatExpCitation",
                                   "LEVEL": "CTMicro_Level",
                                   "IS_BEST": "CTMicro_IsBest",
                                   "CNCI": "CTMicro_CNCI"
                                  }, inplace=True)
# remove_micro_nest.head()

In [ ]:
## Replace all occurring digits in the strings with nothing"

remove_micro_nest['CTMacro_Subj'] = remove_micro_nest['CTMacro_Subj'].str.replace('\d+','').str.lstrip("..")
remove_micro_nest['CTMeso_Subj'] = remove_micro_nest['CTMeso_Subj'].str.replace('\d+','').str.lstrip("..")
remove_micro_nest['CTMicro_Subj'] = remove_micro_nest['CTMicro_Subj'].str.replace('\d+','').str.lstrip("..")


In [ ]:
## Prepare the fsm_elements_df by adding a column for assession number

fsm_elements_df["ACCESSION_NUMBER"] = fsm_elements_df["web_of_science"].str.lstrip("WOS:")
fsm_elements_df.head()

In [ ]:
merged_df = pd.merge(fsm_elements_df, remove_micro_nest, how="left", left_on='ACCESSION_NUMBER', right_on='ACCESSION_NUMBER')
merged_df.head()

In [ ]:
## Export file to excel, without the Pandas index, but with the header

merged_df.to_excel("output/merged.xlsx", index=False, header=True)


In [ ]:
## Drop rows that don't have a citation topic

merged_df.dropna(subset=["CTMicro_Subj"], inplace=True)  
new_merged_df = merged_df[merged_df["CTMicro_Subj"].str.contains("No Topic assigned") == False]


new_merged_df.head()

## Resources
## https://www.statology.org/pandas-drop-rows-that-contain-string/

In [ ]:
# iterating the columns
# for col in merged_df.columns:
#     print(col)

new_merged_df.dtypes

In [ ]:
## Group by NetID and sum number of pubs from search results

groupby_df = new_merged_df.groupby('netid').agg({'name': 'first', 
                                             'position' : 'first',
                                             'department': 'first', 
                                             'school': 'first', 
                                             'employee_id': 'first',
                                             'is_current_staff' : 'first',
                                             'is_academic' : 'first',
                                             'ACCESSION_NUMBER': lambda x: list(x),
                                             'CTMacro_Subj':lambda x: list(x),
                                             'CTMeso_Subj':lambda x: list(x),
                                             'CTMicro_Subj':lambda x: list(x)
                                             }).reset_index()
groupby_df.head()

In [ ]:
## Export file to excel, without the Pandas index, but with the header

groupby_df.to_excel("output/groupby.xlsx", index=False, header=True)